<h2 align='center'>Week 3 Part 2</h2>

In [42]:
#importing required libraries
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import numpy as np

## Accessing webpage

In [2]:
#getting data from the webpage
import urllib.request
weburl=urllib.request.urlopen('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

In [3]:
t=weburl.read()
soup=BeautifulSoup(t)

## Extracting data from returned web page

In [4]:
#Extracting column names
l=[]
for i in soup.find_all('th'):
    k=(re.findall('>([A-Z][a-z]*)*',str(i)))
    l.append(k[0])
l=l[0:-1]

In [5]:
#Extracting each column values
m=[]
for i in soup.find_all('tr'):
    m.append(i.text)
m=m[1:-4]
v=[]
k=[]
c=0
for i in m:
    k=[]
    t=i.split('\n')
    if(t[3]=='Not assigned'):
        continue
    k.append(t[1])
    k.append(t[3])
    k.append(t[5])
    v.append(k)

## Creating Dataframe

In [6]:
df = pd.DataFrame(v, columns = l) 
df.head()

,Postal,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


### Replacing '/' with ','

In [7]:
df['Neighborhood']=df.apply(lambda x: x['Neighborhood'].replace('/',','),axis=1)

### Displaying size of the dataframe

In [8]:
df.shape

(103, 3)

### Renaming column

In [9]:
df.rename(columns={'Postal':'PostalCode'},inplace=True)
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"


### Converting it into given output

In [10]:
tdf = pd.DataFrame(columns=["PostalCode", "Borough", "Neighborhood"])
l = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]
for x in l:
    tdf = tdf.append(df[df["PostalCode"]==x], ignore_index=True)
tdf

,PostalCode,Borough,Neighborhood
0,M5G,Downtown Toronto,Central Bay Street
1,M2H,North York,Hillcrest Village
2,M4B,East York,"Parkview Hill , Woodbine Gardens"
3,M1J,Scarborough,Scarborough Village
4,M4G,East York,Leaside
5,M4M,East Toronto,Studio District
6,M1R,Scarborough,"Wexford , Maryvale"
7,M9V,Etobicoke,"South Steeles , Silverstone , Humbergate , Jam..."
8,M9L,North York,Humber Summit
9,M5V,Downtown Toronto,"CN Tower , King and Spadina , Railway Lands , ..."


## Reading latitude and longtitude

In [11]:
cood= pd.read_csv('Geospatial_Coordinates.csv')
cood.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [12]:
cood.rename(columns={'Postal Code':'PostalCode'},inplace=True)
cood.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


## Merging the datasets

In [13]:
df = df.merge(cood, on="PostalCode", how="left")
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.662301,-79.389494


## Displaying the required output

In [14]:
column_names = ["PostalCode", "Borough", "Neighborhood", "Latitude", "Longitude"]
test_df = pd.DataFrame(columns=column_names)

test_list = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

for postcode in test_list:
    test_df = test_df.append(df[df["PostalCode"]==postcode], ignore_index=True)
    
test_df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
1,M2H,North York,Hillcrest Village,43.803762,-79.363452
2,M4B,East York,"Parkview Hill , Woodbine Gardens",43.706397,-79.309937
3,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
4,M4G,East York,Leaside,43.709060,-79.363452
5,M4M,East Toronto,Studio District,43.659526,-79.340923
6,M1R,Scarborough,"Wexford , Maryvale",43.750072,-79.295849
7,M9V,Etobicoke,"South Steeles , Silverstone , Humbergate , Jam...",43.739416,-79.588437
8,M9L,North York,Humber Summit,43.756303,-79.565963
9,M5V,Downtown Toronto,"CN Tower , King and Spadina , Railway Lands , ...",43.628947,-79.394420


## Coordinates of Toronto

In [15]:
from geopy.geocoders import Nominatim 

address = 'Toronto'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


## Ctreating Map

In [24]:
#importing library
import folium

tmap = folium.Map(location=[latitude, longitude], zoom_start=11)
for Latitude, Longitude, Borough, Neighborhood in zip(df.Latitude, df.Longitude, df.Borough, df.Neighborhood):
    label = '{}, {}'.format(Neighborhood, Borough)
    label = folium.Popup(label, parse_html=True)
    folium.features.CircleMarker(
        [Latitude, Longitude],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(tmap)
tmap

### Extracting neighborhoods in Toronto

In [18]:
b=[]
for x in df.Borough.unique():
    if "Toronto"in x:
        b.append(x)
b

['Downtown Toronto', 'East Toronto', 'West Toronto', 'Central Toronto']

In [19]:
# Extracting rows from dataframe that are a part of toronto
dfb=df[df.Borough.isin (b)]
dfb.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.662301,-79.389494
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [72]:
## Creating map for neighborhood's in Toronto
tmap = folium.Map(location=[latitude, longitude], zoom_start=12)
for Latitude, Longitude, Borough, Neighborhood in zip(dfb.Latitude, dfb.Longitude, dfb.Borough, dfb.Neighborhood):
    label = '{}, {}'.format(Neighborhood, Borough)
    label = folium.Popup(label, parse_html=True)
    folium.features.CircleMarker(
        [Latitude, Longitude],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(tmap)
tmap

## Entering Foursquare Credentials

In [25]:
CLIENT_ID = 'EDG23JWF1NHKORFBEDPYYUSBQAXOFCGFZ4ACBYJ5ZFHGOE54'
CLIENT_SECRET = 'J1OV000RXYKPLNL0YRCVSLFD4ICZEJDLYZWPYIPI3JWBTI2Q'
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: EDG23JWF1NHKORFBEDPYYUSBQAXOFCGFZ4ACBYJ5ZFHGOE54
CLIENT_SECRET:J1OV000RXYKPLNL0YRCVSLFD4ICZEJDLYZWPYIPI3JWBTI2Q


## Extracting information about neighboring locations

In [26]:
radius = 500
LIMIT = 20

venues = []

for lat, long, post, borough, neighborhood in zip(dfb['Latitude'], dfb['Longitude'], dfb['PostalCode'], dfb['Borough'], dfb['Neighborhood']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append((
            post, 
            borough,
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

### Creating dataframe from extracted information

In [30]:
vdf = pd.DataFrame(venues)
vdf.columns = ['PostalCode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude', 'VName', 'VLatitude', 'VLongitude', 'Category']
vdf.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,VName,VLatitude,VLongitude,Category
0,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot


### Finding the categories f the different locations

In [31]:
vdf['Category'].value_counts()

Coffee Shop                  44
Café                         42
Restaurant                   25
Park                         24
Italian Restaurant           18
Bakery                       14
Pub                          13
Bar                          11
Pizza Place                  10
Sandwich Place               10
Sushi Restaurant             10
Gastropub                    10
Dessert Shop                 10
Gym                           9
Diner                         9
Thai Restaurant               8
Japanese Restaurant           8
American Restaurant           8
Bookstore                     8
Beer Bar                      8
Farmers Market                7
Brewery                       7
Breakfast Spot                7
Grocery Store                 7
Theater                       6
Cocktail Bar                  6
Gym / Fitness Center          6
Greek Restaurant              6
Mexican Restaurant            6
Burrito Place                 6
                             ..
Taiwanes

hence the area contains a lot of coffee shops

In [36]:
#making a list of all the categories and creating new dataframe
tdf = pd.get_dummies(vdf[['Category']], prefix="", prefix_sep="")
tdf['PostalCode'] = vdf['PostalCode'] 
tdf['Borough'] = vdf['Borough'] 
tdf['Neighborhoods'] = vdf['Neighborhood'] 
tdf.head()

,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,...,Theme Restaurant,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Yoga Studio,PostalCode,Borough,Neighborhoods
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,M5A,Downtown Toronto,"Regent Park , Harbourfront"
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,M5A,Downtown Toronto,"Regent Park , Harbourfront"
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,M5A,Downtown Toronto,"Regent Park , Harbourfront"
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,M5A,Downtown Toronto,"Regent Park , Harbourfront"


In [38]:
tdf = tdf.groupby(["PostalCode", "Borough", "Neighborhoods"]).mean().reset_index()
tdf.head()

,PostalCode,Borough,Neighborhoods,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,M4E,East Toronto,The Beaches,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.00,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.00
1,M4K,East Toronto,"The Danforth West , Riverdale",0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.05
2,M4L,East Toronto,"India Bazaar , The Beaches West",0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00
3,M4M,East Toronto,Studio District,0.0,0.0,0.0,0.0,0.0,0.0,0.05,...,0.0,0.05,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00
4,M4N,Central Toronto,Lawrence Park,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00


### Top venues for each postal code

In [52]:
n=10
c=['PostalCode', 'Borough', 'Neighborhoods','1','2','3','4','5','6','7','8','9','10']
top = pd.DataFrame(columns=c)
top['PostalCode'] = tdf['PostalCode']
top['Borough'] = tdf['Borough']
top['Neighborhoods'] = tdf['Neighborhoods']

for ind in np.arange(tdf.shape[0]):
    r = tdf.iloc[ind, :].iloc[3:]
    rs = r.sort_values(ascending=False)
    top.iloc[ind, 3:] = rs.index.values[0:n]
top

,PostalCode,Borough,Neighborhoods,1,2,3,4,5,6,7,8,9,10
0,M4E,East Toronto,The Beaches,Health Food Store,Neighborhood,Trail,Pub,Yoga Studio,Discount Store,Diner,Dessert Shop,Department Store,Deli / Bodega
1,M4K,East Toronto,"The Danforth West , Riverdale",Greek Restaurant,Italian Restaurant,Ice Cream Shop,Yoga Studio,Bookstore,Brewery,Bubble Tea Shop,Restaurant,Dessert Shop,Pub
2,M4L,East Toronto,"India Bazaar , The Beaches West",Park,Fast Food Restaurant,Movie Theater,Pet Store,Sandwich Place,Italian Restaurant,Ice Cream Shop,Brewery,Pub,Burrito Place
3,M4M,East Toronto,Studio District,Coffee Shop,Café,Cheese Shop,Sandwich Place,Italian Restaurant,Seafood Restaurant,Brewery,Ice Cream Shop,Bakery,Pet Store
4,M4N,Central Toronto,Lawrence Park,Park,Swim School,Bus Line,Cuban Restaurant,Distribution Center,Discount Store,Diner,Dessert Shop,Department Store,Deli / Bodega
5,M4P,Central Toronto,Davisville North,Park,Food & Drink Shop,Sandwich Place,Breakfast Spot,Hotel,Gym,Department Store,Discount Store,Diner,Dessert Shop
6,M4R,Central Toronto,North Toronto West,Clothing Store,Coffee Shop,Sporting Goods Shop,Chinese Restaurant,Miscellaneous Shop,Park,Mexican Restaurant,Café,Rental Car Location,Restaurant
7,M4S,Central Toronto,Davisville,Dessert Shop,Gym,Italian Restaurant,Sushi Restaurant,Café,Seafood Restaurant,Diner,Brewery,Sandwich Place,Pizza Place
8,M4T,Central Toronto,"Moore Park , Summerhill East",Trail,Playground,Tennis Court,Yoga Studio,Creperie,Discount Store,Diner,Dessert Shop,Department Store,Deli / Bodega
9,M4V,Central Toronto,"Summerhill West , Rathnelly , South Hill , For...",Pub,Coffee Shop,Sushi Restaurant,Light Rail Station,Bank,Health & Beauty Service,Bagel Shop,Sports Bar,Pizza Place,Supermarket


## K-Means 

In [54]:
from sklearn.cluster import KMeans #importing library

In [68]:
kclusters = 5

toronto_grouped_clustering = tdf.drop(["PostalCode", "Borough", "Neighborhoods"], 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)
kmeans.labels_[0:10]

array([3, 3, 3, 0, 2, 3, 3, 0, 4, 0], dtype=int32)

In [69]:
toronto_merged = dfb.copy()

# adding labels
toronto_merged["Cluster Labels"] = kmeans.labels_
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.drop(["Borough", "Neighborhoods"], 1).set_index("PostalCode"), on="PostalCode")
#added latitude and longtitude
print(toronto_merged.shape)
toronto_merged.head() 

(39, 16)


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1,2,3,4,5,6,7,8,9,10
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636,3,Coffee Shop,Breakfast Spot,Farmers Market,Historic Site,Pub,Dessert Shop,Gym / Fitness Center,Spa,Bakery,Performing Arts Venue
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.662301,-79.389494,3,Coffee Shop,Diner,Hobby Shop,Park,Mexican Restaurant,Creperie,Burrito Place,Burger Joint,Juice Bar,Discount Store
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,3,Café,Coffee Shop,Movie Theater,Sandwich Place,Diner,Ramen Restaurant,Electronics Store,Plaza,Burger Joint,Burrito Place
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Gastropub,Hotel,Japanese Restaurant,Coffee Shop,Restaurant,BBQ Joint,Food Truck,Gym,Creperie,Cosmetics Shop
19,M4E,East Toronto,The Beaches,43.676357,-79.293031,2,Health Food Store,Neighborhood,Trail,Pub,Yoga Studio,Discount Store,Diner,Dessert Shop,Department Store,Deli / Bodega


In [70]:
print(toronto_merged.shape)
toronto_merged.sort_values(["Cluster Labels"], inplace=True)
toronto_merged

(39, 16)


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1,2,3,4,5,6,7,8,9,10
62,M5N,Central Toronto,Roselawn,43.711695,-79.416936,0,Garden,Home Service,Yoga Studio,Cuban Restaurant,Distribution Center,Discount Store,Diner,Dessert Shop,Department Store,Deli / Bodega
74,M5R,Central Toronto,"The Annex , North Midtown , Yorkville",43.672710,-79.405678,0,Café,Coffee Shop,Sandwich Place,Pizza Place,Liquor Store,Pub,Cosmetics Shop,BBQ Joint,Pharmacy,Park
68,M5P,Central Toronto,Forest Hill North & West,43.696948,-79.411307,0,Park,Trail,Jewelry Store,Sushi Restaurant,Bus Line,Coffee Shop,College Gym,Clothing Store,Discount Store,Diner
67,M4P,Central Toronto,Davisville North,43.712751,-79.390197,0,Park,Food & Drink Shop,Sandwich Place,Breakfast Spot,Hotel,Gym,Department Store,Discount Store,Diner,Dessert Shop
61,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,0,Park,Swim School,Bus Line,Cuban Restaurant,Distribution Center,Discount Store,Diner,Dessert Shop,Department Store,Deli / Bodega
54,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Coffee Shop,Café,Cheese Shop,Sandwich Place,Italian Restaurant,Seafood Restaurant,Brewery,Ice Cream Shop,Bakery,Pet Store
83,M4T,Central Toronto,"Moore Park , Summerhill East",43.689574,-79.383160,0,Trail,Playground,Tennis Court,Yoga Studio,Creperie,Discount Store,Diner,Dessert Shop,Department Store,Deli / Bodega
47,M4L,East Toronto,"India Bazaar , The Beaches West",43.668999,-79.315572,0,Park,Fast Food Restaurant,Movie Theater,Pet Store,Sandwich Place,Italian Restaurant,Ice Cream Shop,Brewery,Pub,Burrito Place
43,M6K,West Toronto,"Brockton , Parkdale Village , Exhibition Place",43.636847,-79.428191,0,Café,Coffee Shop,Breakfast Spot,Grocery Store,Stadium,Bakery,Burrito Place,Pet Store,Performing Arts Venue,Gym
84,M5T,Downtown Toronto,"Kensington Market , Chinatown , Grange Park",43.653206,-79.400049,0,Café,Mexican Restaurant,Vietnamese Restaurant,Coffee Shop,Fish Market,Bakery,Farmers Market,Caribbean Restaurant,Cheese Shop,Arts & Crafts Store


In [71]:
import matplotlib.cm as cm
import matplotlib.colors as colors
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, post, bor, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['PostalCode'], toronto_merged['Borough'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup('{} ({}): {} - Cluster {}'.format(bor, post, poi, cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters